In [ ]:
from kubernetes import client 
from kserve import KServeClient
from kserve import constants
from kserve import utils
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec

In [ ]:
name = "logistical"
namespace = utils.get_default_target_namespace()
kserve_version='v1beta1'
api_version = constants.KSERVE_GROUP + '/' + kserve_version
isvc2 = V1beta1InferenceService(
    api_version=api_version,
    kind=constants.KSERVE_KIND,
    metadata=client.V1ObjectMeta(
        name=name,
        namespace=namespace,
        annotations={'sidecar.istio.io/inject': 'false'}
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            service_account_name="s3-service-account",
            sklearn=V1beta1SKLearnSpec(
                storage_uri="s3://dvcbucketews/logreg.pkl"
            )
        )
    )
)

In [ ]:
KServe = KServeClient()
KServe.create(isvc2)

In [ ]:
KServe = KServeClient()

import requests

namespace = utils.get_default_target_namespace()


isvc_status = KServe.get(name, namespace=namespace)
print(isvc_status)

In [ ]:
try:
    model_endpoint = isvc_status['status']['address']['url']
    print(f"Model endpoint: {model_endpoint}")
except KeyError:
    print("Error: Unable to retrieve the model endpoint.")

In [ ]:
# steps: 
#1. rename the old model to its original name (by cp then delete)
#2. rename the new model to mode (by cp and delete)

In [ ]:
import boto3


s3 = boto3.client('s3')

# Define the bucket name, original object key, and new object key
bucket_name = 'dvcbucketews'
original_key = 'logreg.pkl'
new_key = 'logreg2.pkl'

# Copy the object
s3.copy_object(
    Bucket=bucket_name,
    CopySource={'Bucket': bucket_name, 'Key': original_key},
    Key=new_key
    
)

# Delete the original object
s3.delete_object(Bucket=bucket_name, Key=original_key)

print(f"Object renamed from {original_key} to {new_key}")

In [ ]:
s3.delete_object(Bucket=bucket_name, Key=original_key)
